In [1]:
import sys
    
import numpy as np

from scipy.optimize import curve_fit

try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.io as pio
    from plotly.offline import iplot, init_notebook_mode

except ModuleNotFoundError or ImportError:
    !{sys.executable} -m pip install plotly
    
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.io as pio
    from plotly.offline import iplot, init_notebook_mode

pio.renderers.default = 'notebook'
pio.templates.default = "seaborn"

import ipywidgets as widgets

try:
    import nPDyn
    from nPDyn.models.Model import Model
    from nPDyn.models.Presets import (lorentzian, gaussian, delta, conv_delta, 
                                      conv_lorentzian_gaussian, conv_lorentzian_lorentzian)
except ModuleNotFoundError or ImportError:
    !{sys.executable} -m pip install git+https://github.com/kpounot/nPDyn@dev#egg=nPDyn

    import nPDyn
    from nPDyn.models.Model import Model
    from nPDyn.models.Presets import (lorentzian, gaussian, delta, conv_delta, 
                                      conv_lorentzian_gaussian, conv_lorentzian_lorentzian)

## Import some data

In [2]:
data = nPDyn.Dataset()

data.importRawData('../../package/tests/sample_data/vanadium/', 'IN16B', 'QENS')

data.binAll(5)

## Using all q-values for the model (here, a pseudo-Voigt profile)

In [3]:
X = data.datasetList[0].data.energies

qVals = data.datasetList[0].data.qVals

params = {'scale': np.zeros((qVals.size, 1)) + 1, 
          'a':np.zeros((qVals.size, 1)) + 0.8, 
          'gauWidth': np.zeros((qVals.size, 1)) + 0.3, 
          'lorWidth': np.zeros((qVals.size, 1)) + 0.25, 
          'shift': np.zeros((qVals.size, 1)) + 0.0, 
          'bkgd': np.zeros((qVals.size, 1)) + 0.001}

m = Model(params=params, xVar=X)

m.eisfComponents['Lorentzian'] = (lorentzian, lambda p: [p['scale'] * (1 - p['a']), p['lorWidth'], p['shift']])
m.eisfComponents['Gaussian'] = (gaussian, lambda p: [p['scale'] * p['a'], p['gauWidth'], p['shift']])
m.bkgdComponents['bkgd'] = (lambda x, bkgd: np.zeros_like(x) + bkgd, lambda p: [p['bkgd']])

m.getModel()


array([[0.00101895, 0.00101933, 0.00101973, ..., 0.00102014, 0.00101973,
        0.00101933],
       [0.00101895, 0.00101933, 0.00101973, ..., 0.00102014, 0.00101973,
        0.00101933],
       [0.00101895, 0.00101933, 0.00101973, ..., 0.00102014, 0.00101973,
        0.00101933],
       ...,
       [0.00101895, 0.00101933, 0.00101973, ..., 0.00102014, 0.00101973,
        0.00101933],
       [0.00101895, 0.00101933, 0.00101973, ..., 0.00102014, 0.00101973,
        0.00101933],
       [0.00101895, 0.00101933, 0.00101973, ..., 0.00102014, 0.00101973,
        0.00101933]])

In [4]:
qSel = widgets.SelectionSlider(
    options=qVals.round(2),
    value=qVals.round(2)[0],
    description='q-value',
    continuous_update=True,
)


fig = go.FigureWidget(layout=go.Layout(xaxis=dict(title='$\\rm \hbar\omega~ [\mu eV]$'),
                                 yaxis=dict(title='$\\rm S(q,\omega)$', type='log')))

qIdx = np.argmin((data.datasetList[0].data.qVals - qSel.value)**2)

fig.add_trace(go.Scatter(x=data.datasetList[0].data.energies, 
                         y=data.datasetList[0].data.intensities[0, qIdx], 
                         error_y=dict(type='data', array=data.datasetList[0].data.errors[0, qIdx]),
                         mode='markers', name="exp"))
fig.add_trace(go.Scatter(x=X, y=m.getModel()[qIdx], name="model"))


def update(val):
    qIdx = np.argmin((data.datasetList[0].data.qVals - qSel.value)**2)

    with fig.batch_update():
        fig.data[1].y = m.getModel()[qIdx]
        fig.data[0].y = data.datasetList[0].data.intensities[0, qIdx]
        fig.data[0].error_y = dict(type='data', array=data.datasetList[0].data.errors[0, qIdx])
    
              
qSel.observe(update, 'value')
              
widgets.VBox([qSel, fig])

## Fitting the data and updating the parameters

In [5]:
pList = m._paramsToList()

popt, perr = curve_fit(lambda x, *p: m.getModel(x, params=p).flatten(), 
                       np.zeros_like(m.getModel()) + X, 
                       data.datasetList[0].data.intensities.flatten(),
                       p0=pList,
                       sigma=data.datasetList[0].data.errors.flatten(),
                       bounds=(-2., 10),
                       method='trf')

In [6]:
m.params = m._listToParams(popt)

## Plotting the optimized model

In [7]:
qSel = widgets.SelectionSlider(
    options=qVals.round(2),
    value=qVals.round(2)[0],
    description='q-value',
    continuous_update=True,
)


fig = go.FigureWidget(layout=go.Layout(xaxis=dict(title='$\\rm \hbar\omega~ [\mu eV]$'),
                                 yaxis=dict(title='$\\rm S(q,\omega)$', type='log', range=[-5, -0.8])))

qIdx = np.argmin((data.datasetList[0].data.qVals - qSel.value)**2)

fig.add_trace(go.Scatter(x=data.datasetList[0].data.energies, 
                         y=data.datasetList[0].data.intensities[0, qIdx], 
                         error_y=dict(type='data', array=data.datasetList[0].data.errors[0, qIdx]),
                         mode='markers', name="exp"))

fig.add_trace(go.Scatter(x=X, y=m.getModel()[qIdx], name="model"))

names, lines = m.getComponents(compType='all')

for line, name in zip(lines, names): 
    fig.add_trace(go.Scatter(x=X, y=line[qIdx], name=name, line=dict(dash='dot')))

def update(val):
    qIdx = np.argmin((data.datasetList[0].data.qVals - qSel.value)**2)
    
    with fig.batch_update():
        fig.data[1].y = m.getModel()[qIdx]
        fig.data[0].y = data.datasetList[0].data.intensities[0, qIdx]
        fig.data[0].error_y = dict(type='data', array=data.datasetList[0].data.errors[0, qIdx])
            
        idx=2
        for line, name in zip(lines, names): 
            fig.data[idx].y = line[qIdx] 
            fig.data[idx].name = name
            idx += 1

    
              
qSel.observe(update, 'value')
              
widgets.VBox([qSel, fig])